# Model Setup
Model 1 takes in img outputs moving probability
Model 2 takes in a sequence of 16 moving probs outputs falling probability

In [1]:
import os

import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')
import tensorflow_datasets as tfds

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import layers
import datetime

import matplotlib.pyplot as plt

import cv2

/home/jessica/VIA-Fall-Detection/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jessica/VIA-Fall-Detection/.venv/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/home/jessica/VIA-Fall-Detection/.venv/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Ten

GPU limitation

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 2GB of memory on the first GPU
    try:
        tf.config.set_logical_device_configuration(gpus[0], [tf.config.LogicalDeviceConfiguration(memory_limit=2048)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


2023-06-07 15:56:24.269537: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-07 15:56:24.292609: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-07 15:56:24.292815: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-07 15:56:24.293678: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

# Building the Models

## Model 1

### Train

In [4]:
data = DataLoader.from_folder(r"./datasets/model1_data")
train_data, test_data = data.split(0.9)

INFO:tensorflow:Load image with size: 4362, num_label: 2, labels: Moving, still.


2023-06-07 15:56:34.684984: I tensorflow/stream_executor/cuda/cuda_driver.cc:739] failed to allocate 2.00G (2147483648 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-06-07 15:56:34.685304: I tensorflow/stream_executor/cuda/cuda_driver.cc:739] failed to allocate 1.80G (1932735232 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-06-07 15:56:34.685607: I tensorflow/stream_executor/cuda/cuda_driver.cc:739] failed to allocate 1.62G (1739461632 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-06-07 15:56:34.685952: I tensorflow/stream_executor/cuda/cuda_driver.cc:739] failed to allocate 1.46G (1565515520 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-06-07 15:56:34.686341: I tensorflow/stream_executor/cuda/cuda_driver.cc:739] failed to allocate 1.31G (1408964096 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-06-07 15:56:34.686641: I tensorflow/stream_executor/cuda/cuda_driver.cc:739] failed to allocate 1.18

In [6]:
model = image_classifier.create(train_data, use_augmentation=True)

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hub_keras_layer_v1v2_1 (Hub  (None, 1280)             3413024   
 KerasLayerV1V2)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 1280)              0         
                                                                 
 dense_1 (Dense)             (None, 2)                 2562      
                                                                 
Total params: 3,415,586
Trainable params: 2,562
Non-trainable params: 3,413,024
_________________________________________________________________
None
INFO:tensorflow:Use default resize_bicubic.


INFO:tensorflow:Use default resize_bicubic.


INFO:tensorflow:Use customized resize method bilinear


INFO:tensorflow:Use customized resize method bilinear


Epoch 1/5


2023-06-07 16:00:24.239983: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8600
2023-06-07 16:00:24.640686: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


122/122 [==============================] - 6s 28ms/step - loss: 0.4071 - accuracy: 0.8581
Epoch 2/5
122/122 [==============================] - 3s 28ms/step - loss: 0.3345 - accuracy: 0.9185
Epoch 3/5
122/122 [==============================] - 3s 28ms/step - loss: 0.3352 - accuracy: 0.9191
Epoch 4/5
122/122 [==============================] - 4s 29ms/step - loss: 0.3138 - accuracy: 0.9344
Epoch 5/5
122/122 [==============================] - 4s 29ms/step - loss: 0.3179 - accuracy: 0.9301


### Evaluate & Export

In [13]:
loss, accuracy = model.evaluate(test_data)
# model.summary()

10/10 [==============================] - 1s 29ms/step - loss: 0.2689 - accuracy: 0.9623


In [ ]:
model.export(export_dir='.')

In [ ]:
### QUANTIZE
# method 1: save as savedmodel then load and quantize like model2 did
# method 2: change datatype before saving
'''
def representative_dataset():
  for d in test_data:
    # d = np.expand_dims(d, axis=0)
    yield [tf.dtypes.cast(d, tf.float32)]

# print(dataset.cardinality().numpy())
# print(tf.shape(dataset))
# model.summary()

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_quant_model2 = converter.convert()
with open('model2_quant', 'wb') as f: f.write(tflite_quant_model2)
'''

### Batch predicting multiple vids

In [ ]:
falling_paths = ["./datasets/model1_vids/splitted_for_model2/resized_IMG0480_falling.mp4",
                 "./datasets/model1_vids/splitted_for_model2/resized_IMG0484_falling.mp4",
                 "./datasets/model1_vids/splitted_for_model2/resized_IMG0485_falling.mp4",
                 "./datasets/model1_vids/splitted_for_model2/resized_IMG1614_falling.mp4",
                 "./datasets/model1_vids/splitted_for_model2/resized_IMG1615_falling.mp4",
                 "./datasets/model1_vids/splitted_for_model2/resized_IMG1616_falling.mp4"]

default_paths = ["./datasets/model1_vids/splitted_for_model2/resized_IMG0480_default.MOV",
                 "./datasets/model1_vids/splitted_for_model2/resized_IMG0484_default.MOV",
                 "./datasets/model1_vids/splitted_for_model2/resized_IMG0485_default.MOV",
                 "./datasets/model1_vids/splitted_for_model2/resized_IMG1614_default.MOV",
                 "./datasets/model1_vids/splitted_for_model2/resized_IMG1615_default.MOV",
                 "./datasets/model1_vids/splitted_for_model2/resized_IMG1616_default.MOV",
                 "./datasets/model2_vids/resized_jess_jump.MOV",
                 "./datasets/model2_vids/resized_jess_pickup.MOV",
                 "./datasets/model2_vids/resized_jess_run1.MOV",
                 "./datasets/model2_vids/resized_jess_run2.MOV",
                 "./datasets/model2_vids/resized_jess_run3.MOV",
                 "./datasets/model2_vids/resized_jess_run4.MOV",
                 "./datasets/model2_vids/resized_jess_walk1.MOV",
                 "./datasets/model2_vids/resized_jess_walk2.MOV",
                 "./datasets/model2_vids/resized_jess_walk3.MOV",
                 "./datasets/model2_vids/resized_jess_walk4.MOV"]

In [ ]:
def batchPredict(path):
    cap = cv2.VideoCapture(path)
    probs = []
    while True:
        ret, frame = cap.read() 
        if not ret: break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_rgb = np.expand_dims(frame_rgb, axis=0)

        interpreter = tf.lite.Interpreter(model_path="model.tflite")
        interpreter.allocate_tensors()
        output = interpreter.get_output_details()
        input = interpreter.get_input_details()
        output_index = output[0]['index']
        input_index = input[0]['index']

        interpreter.set_tensor(input_index, frame_rgb)
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_index)
        output_data = output_data[0]
        output_probs = tf.nn.softmax(output_data.astype(float))

        predicted_index = np.argmax(output_data)
        class_labels = ["Moving", "Still"]
        predicted_class = class_labels[predicted_index]        
        prob = np.around(max(output_probs.numpy()), decimals = 2)

        if predicted_class == "Still": probs.append(np.subtract(1, prob))
        else: probs.append(prob)
    return probs

In [ ]:
def batchPredictMultVids(vid_path_lst):
    record = []
    for vid in vid_path_lst:
        vid_probs = batchPredict(vid)
        print(vid)
        for i in range(len(vid_probs)-15):
            record.append(vid_probs[i:i+16])
            for j in range(i, i+16):
                print(vid_probs[j], end=" ")
            print()
        print()
    return record

In [ ]:
falling = batchPredictMultVids(falling_paths)
default = batchPredictMultVids(default_paths)

## Model 2

### Train

In [3]:
default_data = np.loadtxt('./datasets/model2_data/default.txt')
falling_data = np.loadtxt('./datasets/model2_data/falling.txt')

In [4]:
mu, sig = 0, 0.001
copy_count = len(default_data)-len(falling_data)
oldLen = len(falling_data)
rowi = 0

# limit = lambda x : max(min(x,1), 0)

def limit(x):
    if x > 1: return 1 - (x-1)
    elif x < 0: return abs(x)
    return x

newLimit = np.vectorize(limit)

for i in range(copy_count):
    row = falling_data[rowi] + np.random.normal(mu, sig, 16)
    row = newLimit(row)

    falling_data = np.vstack([falling_data, row])
    rowi += 1
    if rowi >= oldLen: rowi = 0

print(falling_data.shape, default_data.shape)

(6270, 16) (6270, 16)


In [5]:
inputs = np.concatenate((default_data, falling_data))
outputs = np.concatenate((np.zeros(len(default_data)), np.ones(len(falling_data))))

# Convert inputs and outputs to TensorFlow Dataset
dataset = tf.data.Dataset.from_tensor_slices((inputs, outputs))
dataset = dataset.shuffle(len(inputs)).batch(128)

2023-06-07 13:42:17.469144: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-07 13:42:17.487032: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-07 13:42:17.487189: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-07 13:42:17.487643: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [6]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(16,)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
#Tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [9]:
model.fit(dataset, epochs=6,callbacks=[tensorboard_callback])

Epoch 1/6
98/98 [==============================] - 1s 2ms/step - loss: 0.5056 - accuracy: 0.8177
Epoch 2/6
98/98 [==============================] - 1s 6ms/step - loss: 0.3968 - accuracy: 0.8474
Epoch 3/6
98/98 [==============================] - 0s 2ms/step - loss: 0.3849 - accuracy: 0.8500
Epoch 4/6
98/98 [==============================] - 0s 2ms/step - loss: 0.3822 - accuracy: 0.8502
Epoch 5/6
98/98 [==============================] - 0s 2ms/step - loss: 0.3815 - accuracy: 0.8506
Epoch 6/6
98/98 [==============================] - 0s 2ms/step - loss: 0.3805 - accuracy: 0.8511


### Evaluate & Export

In [8]:
### CONVERT TO TFLITE
converter = tf.lite.TFLiteConverter.from_keras_model(model)
model2_tflite = converter.convert()
with open('model2.tflite', 'wb') as f: f.write(model2_tflite)

INFO:tensorflow:Assets written to: /tmp/tmpgy6fdty2/assets


2023-06-07 11:16:44.155686: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2023-06-07 11:16:44.445558: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-06-07 11:16:44.445580: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2023-06-07 11:16:44.446467: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpgy6fdty2
2023-06-07 11:16:44.447205: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2023-06-07 11:16:44.447233: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpgy6fdty2
2023-06-07 11:16:44.449735: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-06-07 11:16:44.489124: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on Saved

In [29]:
### QUANTIZE
def representative_dataset():
  for d in inputs:
    # d = np.expand_dims(d, axis=0)
    yield [tf.dtypes.cast(d, tf.float32)]

# print(dataset.cardinality().numpy())
# print(tf.shape(dataset))
# model.summary()

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_quant_model2 = converter.convert()
with open('model2_quant', 'wb') as f: f.write(tflite_quant_model2)

INFO:tensorflow:Assets written to: /tmp/tmpdvgpu7f5/assets


INFO:tensorflow:Assets written to: /tmp/tmpdvgpu7f5/assets
/home/jessica/VIA-Fall-Detection/.venv/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-06-07 15:22:06.987031: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-06-07 15:22:06.987052: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2023-06-07 15:22:06.987225: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpdvgpu7f5
2023-06-07 15:22:06.987830: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2023-06-07 15:22:06.987841: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpdvgpu7f5
2023-06-07 15:22:06.989672: I tensorflow/cc/sav

In [42]:
# loss, acc = model.evaluate(X_test, y_test)
# model.summary()
model.save("model2")

INFO:tensorflow:Assets written to: model2/assets


# Predict

## Reformat Input

In [47]:
def resize_video(input_path, output_path, width, height):
  # Open the video file
  video = cv2.VideoCapture(input_path)

  # Get the original video's width and height
  original_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
  original_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
  
  # Create a VideoWriter object to save the resized video
  fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Codec for the output video
  fps = video.get(cv2.CAP_PROP_FPS)
  writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
  
  while True:
    # Read a frame from the original video
    ret, frame = video.read()
    if not ret: break
    # Resize the frame to the desired width and height
    resized_frame = cv2.resize(frame, (width, height))
    # Write the resized frame to the output video file
    writer.write(resized_frame)

# Release the video capture and writer objects
  video.release()
  writer.release()

In [62]:
# Resize vid
video_path = "compilation_cut.mp4"
output_path = "compilation_cut_resized.mp4"
target_width = 224
target_height = 224

#resize_video(video_path, output_path, target_width, target_height)

# Load resized vid
cap = cv2.VideoCapture(output_path)
# frame_rate = 30
# cap.set(cv2.CAP_PROP_FPS, frame_rate) DOESN'T WORK
print(cap.get(cv2.CAP_PROP_FPS))

29.97


## Pass into model 1

In [63]:
# Initialize list to store the frame classifications
frame_classifications = []
moving_prob = []
# Loop through the frames of the video (need to change to 30 fps)
while True:
    ret, frame = cap.read() 
    #just need to figure out if this is 30 fps

    if not ret: # Break the loop if the video has ended
        break

    ''' Formulate Input Data (frame_rgb) '''
    # Convert the frame to RGB format
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Make frame input data and ensure its type matches the model
    frame_rgb = np.expand_dims(frame_rgb, axis=0)

    ''' Classify the Frame '''
    interpreter = tf.lite.Interpreter(model_path="model.tflite")
    interpreter.allocate_tensors()

    # get_output_details() and get_input_details() return list of dictionaries of tensor details
    # keys: name, index, shape, shape_signature, dtype, quantization, ...
    # len(input) = len(output) = 1, so access the first element
    output = interpreter.get_output_details()
    input = interpreter.get_input_details()
    output_index = output[0]['index']
    input_index = input[0]['index']

    # set input -> invoke -> access output
    interpreter.set_tensor(input_index, frame_rgb)
    interpreter.invoke()

    output_data = interpreter.get_tensor(output_index)
    # If the output_data shape is (batch_size, num_classes), select the first frame
    output_data = output_data[0]

    # Convert each entry into probability
    output_probs = tf.nn.softmax(output_data.astype(float))

    # Find the index of the highest probability
    predicted_index = np.argmax(output_data)

    # Assuming you have a list of class labels corresponding to the model's output classes
    class_labels = ["Moving", "Still"]

    # Get the predicted class label
    predicted_class = class_labels[predicted_index]

    # Print the predicted class label
    # print("Predicted Class:", predicted_class)
    frame_classifications.append((predicted_class, max(output_probs.numpy())))
    
    prob = np.around(max(output_probs.numpy()), decimals = 2)
    if predicted_class == "Still":
        
        moving_prob.append(np.subtract(1, prob))
    else:
        moving_prob.append(prob)

## Prepare data for model 2

In [64]:
moving_probs_trimmed = moving_prob[:-(len(moving_prob)%16)]
model2_in = np.array(moving_probs_trimmed).reshape((len(moving_prob)//16, 16))
print(model2_in)

[[0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.  ]
 [0.   0.   0.   0.   0.   1.   1.   1.   1.   1.   1.   1.   1.   1.
  1.   1.  ]
 [0.   0.   0.   0.   0.   0.   0.   1.   1.   1.   1.   1.   1.   1.
  1.   0.  ]
 [0.   0.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.
  1.   1.  ]
 [1.   1.   1.   1.   1.   1.   1.   1.   1.   0.   0.   1.   0.   1.
  1.   1.  ]
 [1.   1.   1.   1.   1.   0.   0.   0.   0.   0.   0.   0.   1.   1.
  1.   1.  ]
 [1.   1.   1.   1.   1.   0.   0.   1.   1.   1.   0.   1.   1.   0.
  1.   1.  ]
 [0.   0.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.
  1.   1.  ]
 [1.

## Pass into Model 2 and get final prediction

In [65]:
model2 = tf.keras.models.load_model("model2")

In [66]:
vid_preds = model2.predict(model2_in)

In [67]:
threshold = 0.88
bools = vid_preds.reshape((1, len(vid_preds))) > threshold
print(bools)
print(vid_preds.reshape((1, len(vid_preds))))

[[False False False False False False False False False False False False
   True  True False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False  True False  True  True False
  False False False False False False False False False False False False
  False False]]
[[0.09045138 0.09045138 0.09045138 0.09045138 0.09045138 0.5739342
  0.30349308 0.7925048  0.7975426  0.51072216 0.7444853  0.7925048
  0.929349   0.929349   0.8644172  0.09045138 0.15479052 0.29671496
  0.09045138 0.15544447 0.09045138 0.09045138 0.09045138 0.09045138
  0.09045138 0.17014171 0.12607063 0.09045138 0.09045138 0.19399568
  0.09045138 0.09045138 0.16418059 0.14767478 0.15647146 0.12583217
  0.09045138 0.09045138 0.18011312 0.09045138 0.11525431 0.17111975
  0.26666683 0.929349   0.80626893 0.929349   0.929349   0.2670536
  0.16497837 0.09045138 0.09045138 0.09045138 0.09045138 0.09045138
  0.0